In [1]:
import pandas as pd
import numpy as np
import gzip, pickle, pickletools
import time

In [2]:
# Archivo donde se calcularon los pesos para cada prestamo

# Para colab
# df_prestamos = pd.read_json('/content/drive/MyDrive/UAQUE/TABLA_JOIN.json')

# Para local
df_prestamos = pd.read_json('../data/TABLA_JOIN.json')

In [3]:
df_temas = pd.read_json('../data/Libros.json')

# Funciones auxiliares

In [4]:
#Para obtener el nombre de un material a partir del título
def itemIdToMaterial(id):
    return df_prestamos.loc[df_prestamos['Llave_x'] == id]['Titulo'].unique()

In [5]:

def itemIdToDewey(id):
    return (df_prestamos.loc[df_prestamos['Llave_x'] == id]['Dewey'].unique())[0]

In [6]:
def itemIdToThemes(id):
    tematicas = df_temas.loc[df_temas['Llaves'] == str(id)]['Tema 670'].values
    if len(tematicas) == 0:
        return []
    else:
        return tematicas

In [7]:
# Función para obtener el diccionario de Deweys
import json
from collections import defaultdict

def getAllDeweys():
    with open('../data/deweys.json') as json_file:
        data = json.load(json_file)
    return defaultdict(list, data)

In [8]:
# Función para obtener el diccionario de Años de publicación
import json
from collections import defaultdict

def getAllPublishedYears():
    with open('../data/years.json') as json_file:
        data = json.load(json_file)
    return defaultdict(int, data)

In [9]:
# Función para obtener el diccionario de temáticas
import json
from collections import defaultdict

def getAllThemes():
    with open('../data/themes.json') as json_file:
        data = json.load(json_file)
    return defaultdict(list, data)

In [10]:

def generateClusterInfo(cluster_eval):
    dfP = df_pesos_cluster_temas.copy()
    user_p = dfP.loc[dfP['cluster'] == cluster_eval]
    tematicas =[]
    for llave in user_p['Llaves'].unique():
        tematicas.append(itemIdToThemes((llave)))

    user_p['Tematicas'] = tematicas
    display(user_p)
    print((dfP.loc[dfP['cluster'] == cluster_eval]['Dewey'].unique()))

In [11]:
def generateRecsInfopreds(preds):
    recs = preds
    data_recs_asr = []
    for ids in recs:
        data_recs_asr.append({'Titulo':itemIdToMaterial(ids), 
                         #'Rating':ratings[1], 
                         'Dewey':itemIdToDewey(ids), 
                         'Tematicas':itemIdToThemes(ids),
                          'Llave': ids})

    df_recs_asr = pd.DataFrame(data_recs_asr)
    display(df_recs_asr)
    print((df_recs_asr['Dewey'].unique()))

In [12]:
def loadSimilarities():
    return np.load('similarities_rec.npy')

In [13]:
import sys
sys.path.append('./')

In [14]:
from sar_recommeder import SarRecommeder

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [15]:
df_pesos_cluster_temas = pd.read_json('../data/pesos_cluster_temas.json')

In [16]:
df_pesos_cluster_deweys = pd.read_json('../data/pesos_cluster_deweys.json')

# SAR Clusters Temáticas

In [17]:
sar_predictor_temas = SarRecommeder(df_pesos_cluster_temas, n=10000)

In [18]:
# Entrenando modelo
sar_predictor_temas.fit()

2021-09-29 15:59:40,735 INFO     Collecting user affinity matrix
2021-09-29 15:59:40,737 INFO     Calculating time-decayed affinities
2021-09-29 15:59:40,745 INFO     Creating index columns
2021-09-29 15:59:40,754 INFO     Calculating normalization factors
2021-09-29 15:59:40,760 INFO     Building user affinity sparse matrix
2021-09-29 15:59:40,761 INFO     Calculating item co-occurrence
2021-09-29 15:59:40,881 INFO     Calculating item similarity
2021-09-29 15:59:40,881 INFO     Using jaccard based similarity
2021-09-29 15:59:41,399 INFO     Done training


In [19]:
# Obteniendo Predicciones
top_k_temas = sar_predictor_temas.predict()

2021-09-29 15:59:41,403 INFO     Calculating recommendation scores


In [20]:
# sar_predictor_temas
#sar_predictor_temas.evaluate_model(top_k_temas)

In [21]:
# Prediciendo para un usuario 
cluster_eval = 92
preds_temas = sar_predictor_temas.predict_for_cluster(cluster=cluster_eval)

In [22]:
# Info cluster
generateClusterInfo(cluster_eval)

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Llaves,cluster,Dewey,Peso del prestamos,timestamp,Tematicas
1692,255776,92,190,0.250,2018-08-03,[]
1695,255779,92,190,0.250,2019-03-19,"[[filosofía, francés]]"
1701,255783,92,800,0.250,2019-02-13,"[[literatura, moral]]"
1850,256000,92,190,0.125,2019-05-22,"[[filosofía, alemán], [filosofía, alemán]]"
1864,256034,92,130,0.125,2018-11-08,[]
...,...,...,...,...,...,...
179612,885340,92,840,1.000,2019-11-19,"[[poesía, francés]]"
179635,885617,92,840,0.500,2019-01-21,"[[cuento, francés]]"
182378,908913,92,270,0.250,2019-01-21,"[[cristianismo, historia]]"
183156,915537,92,110,0.750,2019-02-06,"[[filosofía, naturaleza]]"


['190' '800' '130' '880' '740' '110' '140' '180' '840' '260' '710' '810'
 '720' '860' '330' '920' '400' '750' '150' '890' '370' '820' '300' '320'
 '780' '170' '520' '590' '20' '500' '120' '830' '0' '460' 'Video' '700'
 '570' '900' '580' '850' '440' '270' '340' '610' '980']


In [23]:
# Info Recomendaciones
generateRecsInfopreds(preds_temas)

,Titulo,Dewey,Tematicas,Llave
0,"[Obras filosoficas Discurso del metodo, Tratad...",190,"[[filosofía, francés]]",272620
1,[La caida Albert Camus ; traducción Alberto Lu...,840,"[[novela, francés]]",279000
2,[Fragmentos Heraclitus ; traducción de Luis Fe...,180,"[[filósofo, griego]]",265911
3,[El exilio y el reino relatos Albert Camus ; t...,840,"[[novela, francés]]",279014
4,[Descartes y la filosofía ],190,"[[filosofía, francés]]",272612
5,[Dynamic figure drawing Burne Hogarth],740,"[[anatomia, artistica]]",259123
6,[Monadología. Discurso de metafisica Gottfried...,110,[],263212
7,[Tragedias Sófocles ; introducción de José S. ...,880,"[[literatura, griego]]",257815


['190' '840' '180' '740' '110' '880']


In [24]:
# Exportando el modelo
#sar_predictor_temas.export_model()

In [25]:
# now_time = time.strftime("%m%d%H%m")
# filepath = "../Models/sar_trained_model_"+str(100)+"k_"+now_time+".pkl"
# with gzip.open(filepath, "wb") as f:
#     pickled = pickle.dumps(sar_predictor_temas.model, protocol=4)
#     optimized_pickle = pickletools.optimize(pickled)
#     f.write(optimized_pickle)

# SAR Clusters Deweys

In [ ]:
sar_predictor_deweys = SarRecommeder(df_pesos_cluster_deweys, n=100000)

In [ ]:
len(sar_predictor_deweys.train)

In [ ]:
# Entrenando modelo
sar_predictor_deweys.fit()

In [ ]:
# Obteniendo Predicciones
top_k_deweys = sar_predictor_deweys.predict()

In [ ]:
# sar_predictor_temas
#sar_predictor_deweys.evaluate_model(top_k_deweys)

In [ ]:
# Prediciendo para un usuario 
cluster_eval = 92
preds_temas = sar_predictor_deweys.predict_for_cluster(cluster=cluster_eval)

In [ ]:
# Info cluster
generateClusterInfo(cluster_eval)

In [ ]:
# Info Recomendaciones
generateRecsInfopreds(preds_temas)

In [ ]:
# Exportando el modelo
#sar_predictor_deweys.export_model()

# Content based KNN Clusters Temáticas

In [26]:
from content_knn_recommender import ContentKNNRecommender

In [27]:
content_predictor_temas = ContentKNNRecommender(df_pesos_cluster_temas, n=40000, scale=5)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [28]:
content_predictor_temas.fit_evaluate()

Evaluando el algoritmo  ContentKNN ...
Calculando Precisión...
Pre fit base
Post fit base
Calculando matriz de similaridad basada en contenido...
...done.
Análisis Completado


Algorithm  RMSE       MAE       
ContentKNN 0.1027     0.0519    

Resumen:

RMSE:      Root Mean Squared Error. Valores más bajos significa mejor precisión.
MAE:       Mean Absolute Error. Valores más bajos significa mejor precisión.


In [29]:
# Prediciendo para un usuario 
cluster_eval = 92
preds_temas_cb = content_predictor_temas.predict_for_cluster(cluster=cluster_eval)

1

Usando el recomendador:  ContentKNN
Computing recommendations...


In [30]:
# Info cluster
generateClusterInfo(cluster_eval)

/opt/anaconda3/envs/rec_base/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Llaves,cluster,Dewey,Peso del prestamos,timestamp,Tematicas
1692,255776,92,190,0.250,2018-08-03,[]
1695,255779,92,190,0.250,2019-03-19,"[[filosofía, francés]]"
1701,255783,92,800,0.250,2019-02-13,"[[literatura, moral]]"
1850,256000,92,190,0.125,2019-05-22,"[[filosofía, alemán], [filosofía, alemán]]"
1864,256034,92,130,0.125,2018-11-08,[]
...,...,...,...,...,...,...
179612,885340,92,840,1.000,2019-11-19,"[[poesía, francés]]"
179635,885617,92,840,0.500,2019-01-21,"[[cuento, francés]]"
182378,908913,92,270,0.250,2019-01-21,"[[cristianismo, historia]]"
183156,915537,92,110,0.750,2019-02-06,"[[filosofía, naturaleza]]"


['190' '800' '130' '880' '740' '110' '140' '180' '840' '260' '710' '810'
 '720' '860' '330' '920' '400' '750' '150' '890' '370' '820' '300' '320'
 '780' '170' '520' '590' '20' '500' '120' '830' '0' '460' 'Video' '700'
 '570' '900' '580' '850' '440' '270' '340' '610' '980']


In [31]:
# Info Recomendaciones
generateRecsInfopreds(preds_temas_cb)

,Titulo,Dewey,Tematicas,Llave
0,[Jesus de Nazaret mensaje e historia Joachim G...,230,[],343629
1,[La colonización de lo imaginario sociedades i...,410,"[[tradición, oral]]",318794
2,[Recent advances in the chemistry of insect co...,630,[],346275
3,"[Mitologias Roland Barthes, ÆSoy el más fuerte...",290,"[[mitología, literatura]]",274878
4,[El tema de la amnistia Arturo Martínez Nateras],300,[],519661
5,[Terapeutica periodontal un estudio completo d...,610,[],329698
6,[Historia contemporanea de Colombia desde la d...,980,"[[none], [none]]",339243
7,[Agoniza Dios? la problemática de Dios en la n...,860,"[[novela, latinoamericano, historia, criticar]]",311438
8,[Metafísica Aristoteles],180,"[[filosofía, antiguo]]",266074
9,[Historia de la filosofía Frederick Charles Co...,100,"[[historia, filosofía], [historia, filosofía],...",255948


['230' '410' '630' '290' '300' '610' '980' '860' '180' '100']


# Content bases KNN Clusters Deweys

In [ ]:
content_predictor_deweys = ContentKNNRecommender(df_pesos_cluster_deweys, n=80000, scale=5)

In [ ]:
content_predictor_deweys.fit_evaluate()

In [ ]:
# Prediciendo para un usuario 
cluster_eval = 92
preds_deweys_cb = content_predictor_deweys.predict_for_cluster(cluster=cluster_eval)

In [ ]:
# Info cluster
generateClusterInfo(cluster_eval)

In [33]:
# Info Recomendaciones
preds_tems_cb = ['30225']
generateRecsInfopreds(preds_tems_cb)

IndexError: index 0 is out of bounds for axis 0 with size 0